# Compress Filter

load epa

In [18]:
import moritz.lindner.masterarbeit.epa.construction.builder.ExtendedPrefixAutomatonBuilder
import moritz.lindner.masterarbeit.epa.construction.builder.SampleEventMapper
import moritz.lindner.masterarbeit.epa.*
import moritz.lindner.masterarbeit.epa.domain.*
import java.io.File

val epa = ExtendedPrefixAutomatonBuilder<Long>()
    .setFile(File("/Users/moritzlindner/programming/Masterarbeit/epa-visualizer/epa/src/test/resources/simple2.xes"))
    .setEventLogMapper(SampleEventMapper())
    .build()

epa

simple2.xes
c1,c2,f1,root,a1,d1,e1,f2,e2,b1
a1,f1,e1,f2,d1,e2,c1,b1,c2
Transition(start=a1, activity=b1, end=b1),Transition(start=a1, activity=c1, end=c1),Transition(start=f1, activity=f2, end=f2),Transition(start=f1, activity=e2, end=e2),Transition(start=root, activity=a1, end=a1),Transition(start=e2, activity=c2, end=c2),Transition(start=e1, activity=f1, end=f1),Transition(start=b1, activity=d1, end=d1),Transition(start=c1, activity=e1, end=e1)
c1:2,c2:3,f1:2,root:0,a1:1,d1:1,e1:2,f2:2,e2:3,b1:1
c1:Event(activity=c1, timestamp=1745625660000, caseIdentifier=2, predecessor=Event(activity=a1, timestamp=1745625600000, caseIdentifier=2, predecessor=null)),Event(activity=c1, timestamp=1745625660000, caseIdentifier=3, predecessor=Event(activity=a1, timestamp=1745625600000, caseIdentifier=3, predecessor=null)),c2:Event(activity=c2, timestamp=1745625900000, caseIdentifier=3, predecessor=Event(activity=e2, timestamp=1745625840000, caseIdentifier=3, predecessor=null)),f1:Event(activity=f1, time

In [19]:
class MarkedState(
    var state: State,
    var isInvalid: Boolean
) {
    override fun toString(): String {
        return "$state: $isInvalid"
    }
}

class SyntheticStates(
    val chains: List<List<State.PrefixState>>,
) {
    private val allChainParts = chains.flatten().toSet()

    val chainByChainStart = chains.map { it.first() to it }.toMap()
    val chainByChainEnd = chains.map { it.last() to it }.toMap()

    val syntheticStateByChain = chains.associateWith { chain ->
        MarkedState(
            state = State.PrefixState(
                from = chain.first().from,
                via = chain.fold(Activity("")) { acc, s -> Activity(acc.name + s.name) }
            ),
            isInvalid = true
        )
    }

    fun isPartOfChain(state: State): Boolean {
        return allChainParts.contains(state)
    }

    override fun toString(): String {
        return chains.map { chains -> chains.joinToString(",") }.joinToString("\n")
    }
}

class Mapping {
    var parentByState = mutableMapOf<State, MarkedState>()
    var childrenByState = mutableMapOf<State, List<MarkedState>>()

    fun markeParentsIfInvalid(chains: SyntheticStates) {
        chains.chains.forEach { chain ->
            parentByState.forEach { state, parent ->
                if (parent.state == chain.last()) {
                    println("marking $parent")
                    parentByState[state]?.isInvalid = true
                }
            }
        }
    }

    fun markChildrenIfInvalid(chains: SyntheticStates) {
        childrenByState.forEach { _, children ->
            children.forEach { child ->
                val isPresent = chains.chainByChainStart[child.state] != null
                if (isPresent) child.isInvalid = true
            }
        }
    }

    fun addParentForState(key: State, value: MarkedState) {
        parentByState.put(key, value)
    }

    fun addChildrenForState(key: State, values: List<MarkedState>) {
        val children = childrenByState.get(key) ?: emptyList()

        childrenByState.put(key, children + values)
    }

    fun addIfNotPresent(state: State) {
        if (childrenByState.contains(state).not()) {
            childrenByState.put(state, emptyList())
        }
    }

    fun <T> mergeSublistsKeepLongest(lists: List<List<T>>): List<List<T>> {
        return lists.filter { currentList ->
            // Keep this list only if no other list contains all of its elements
            lists.none { otherList ->
                otherList != currentList && otherList.containsAll(currentList)
            }
        }
    }

    fun detectChains(): SyntheticStates {
        val chains = childrenByState
            .filter { it.key is State.PrefixState }
            .filter { it.value.size == 1 }
            .map { (state, _) ->
                listOf(state) + followChain(state, emptyList())
            }.map { chain ->
                chain.map { it as State.PrefixState }
            }

        return SyntheticStates(mergeSublistsKeepLongest(chains))
    }

    fun followChain(a: State, acc: List<State>): List<State> {
        if (childrenByState[a] != null && childrenByState[a]!!.isNotEmpty()) {
            val n = childrenByState[a]!!.first()
            return if (childrenByState[a]!!.size == 1) followChain(n.state, acc + n.state)
            else acc
        } else return acc
    }

    override fun toString(): String {
        return "parents:\n${parentByState.map { "${it.key} -> ${it.value}\n" }}\n" +
                "children:\n${childrenByState.map { "${it.key} -> [${it.value.joinToString()}]" }}"
    }

    fun addSyntheticStates(syntheticStates: SyntheticStates) {
        syntheticStates.chains.forEach { chain ->
            val parent = parentByState[chain.first()]!!
            parentByState.put(syntheticStates.syntheticStateByChain[chain]!!.state, parent)

            val children = childrenByState[chain.last()]!!
            childrenByState.put(syntheticStates.syntheticStateByChain[chain]!!.state, children)
        }
    }

    fun removeAllStatesWhichArePartOfChain(syntheticStates: SyntheticStates) {
        val newparentByState = mutableMapOf<State, MarkedState>()
        val newchildrenByState = mutableMapOf<State, List<MarkedState>>()

        parentByState.forEach { state, p ->
            if (syntheticStates.isPartOfChain(state).not()) newparentByState.put(state, p)
        }

        childrenByState.forEach { state, c ->
            if (syntheticStates.isPartOfChain(state).not()) newchildrenByState.put(state, c)
        }

        parentByState = newparentByState
        childrenByState = newchildrenByState
    }

    fun updateParents(syntheticStates: SyntheticStates) {
        parentByState.forEach { state, parent ->
            if (parent.isInvalid) {
                val chain = syntheticStates.chainByChainEnd[parent.state]!!
                val newstate = syntheticStates.syntheticStateByChain[chain]!!
                newstate.isInvalid = false
                parentByState[state] = newstate
            }
        }
    }

    fun updateChildren(syntheticStates: SyntheticStates) {
        childrenByState.forEach { state, children ->
            val update = children.map { child ->
                if (child.isInvalid) {
                    val chain = syntheticStates.chainByChainStart[child.state]
                    val newState = syntheticStates.syntheticStateByChain[chain]!!
                    newState.isInvalid = false
                    newState
                } else child
            }
            childrenByState.put(state, update)
        }
    }

    fun buildNewEpa(epa: ExtendedPrefixAutomaton<Long>): ExtendedPrefixAutomaton<Long> {

        val transitions = childrenByState.flatMap { (state, children) ->
            children.map { child ->
                Transition(
                    start = state,
                    activity = (child.state as State.PrefixState).via,
                    end = child.state
                )
            }
        }

        parentByState.map { (state, parent) ->
            state as State.PrefixState
            state.from = parent.state
        }

        return ExtendedPrefixAutomaton<Long>(
            eventLogName = epa.eventLogName + "compressed",
            states = (listOf(State.Root) + parentByState.keys.toList()).toSet(),
            activities = transitions.map { it.activity }.toSet(),
            transitions = transitions.toSet(),
            partitionByState = emptyMap(),
            sequenceByState = emptyMap()
        )
    }
}

1. Create mapping of epa

In [20]:
val mapping = Mapping()

val childrenByParent = epa.transitions.groupBy { it.start }.mapValues { it.value.map { it.end } }
val parentByChild = epa.transitions.groupBy { it.end }.mapValues { it.value.map { it.start }.first() }

childrenByParent.forEach { state, children ->
    mapping.addChildrenForState(state, children.map { MarkedState(it as State.PrefixState, false) })
}

parentByChild.forEach { state, parent ->
    mapping.addParentForState(state, MarkedState(parent, false))
}

epa.states.forEach { state ->
    mapping.addIfNotPresent(state)
}

mapping

parents:
[b1 -> a1: false
, c1 -> a1: false
, f2 -> f1: false
, e2 -> f1: false
, a1 -> root: false
, c2 -> e2: false
, f1 -> e1: false
, d1 -> b1: false
, e1 -> c1: false
]
children:
[a1 -> [b1: false, c1: false], f1 -> [f2: false, e2: false], root -> [a1: false], e2 -> [c2: false], e1 -> [f1: false], b1 -> [d1: false], c1 -> [e1: false], c2 -> [], d1 -> [], f2 -> []]

detect chains

In [21]:
val chains = mapping.detectChains()
chains

e2,c2
b1,d1
c1,e1,f1

3.1 Marke parent of each state as invalid if the parent-state is a chain end

In [22]:
mapping.markeParentsIfInvalid(chains)
mapping

marking f1: false
marking f1: false


parents:
[b1 -> a1: false
, c1 -> a1: false
, f2 -> f1: true
, e2 -> f1: true
, a1 -> root: false
, c2 -> e2: false
, f1 -> e1: false
, d1 -> b1: false
, e1 -> c1: false
]
children:
[a1 -> [b1: false, c1: false], f1 -> [f2: false, e2: false], root -> [a1: false], e2 -> [c2: false], e1 -> [f1: false], b1 -> [d1: false], c1 -> [e1: false], c2 -> [], d1 -> [], f2 -> []]

3.1 Marke children of each state as invalid if the state is a chain start

In [23]:
mapping.markChildrenIfInvalid(chains)
mapping

parents:
[b1 -> a1: false
, c1 -> a1: false
, f2 -> f1: true
, e2 -> f1: true
, a1 -> root: false
, c2 -> e2: false
, f1 -> e1: false
, d1 -> b1: false
, e1 -> c1: false
]
children:
[a1 -> [b1: true, c1: true], f1 -> [f2: false, e2: true], root -> [a1: false], e2 -> [c2: false], e1 -> [f1: false], b1 -> [d1: false], c1 -> [e1: false], c2 -> [], d1 -> [], f2 -> []]

add new states and add marked state from lookup table for parents and children

In [24]:
mapping.addSyntheticStates(chains)
mapping

parents:
[b1 -> a1: false
, c1 -> a1: false
, f2 -> f1: true
, e2 -> f1: true
, a1 -> root: false
, c2 -> e2: false
, f1 -> e1: false
, d1 -> b1: false
, e1 -> c1: false
, e2c2 -> f1: true
, b1d1 -> a1: false
, c1e1f1 -> a1: false
]
children:
[a1 -> [b1: true, c1: true], f1 -> [f2: false, e2: true], root -> [a1: false], e2 -> [c2: false], e1 -> [f1: false], b1 -> [d1: false], c1 -> [e1: false], c2 -> [], d1 -> [], f2 -> [], e2c2 -> [], b1d1 -> [], c1e1f1 -> [f2: false, e2: true]]

5. Remove all states which are part of chain

In [25]:
mapping.removeAllStatesWhichArePartOfChain(chains)
mapping

parents:
[f2 -> f1: true
, a1 -> root: false
, e2c2 -> f1: true
, b1d1 -> a1: false
, c1e1f1 -> a1: false
]
children:
[a1 -> [b1: true, c1: true], root -> [a1: false], f2 -> [], e2c2 -> [], b1d1 -> [], c1e1f1 -> [f2: false, e2: true]]

6. update invalidated parents by takeing chain as parent where old parent is chain-end

In [26]:
mapping.updateParents(chains)
mapping

parents:
[f2 -> c1e1f1: false
, a1 -> root: false
, e2c2 -> c1e1f1: false
, b1d1 -> a1: false
, c1e1f1 -> a1: false
]
children:
[a1 -> [b1: true, c1: true], root -> [a1: false], f2 -> [], e2c2 -> [], b1d1 -> [], c1e1f1 -> [f2: false, e2: true]]

6. update invalidated children by taking new chain state where children is start

In [27]:
mapping.updateChildren(chains)
mapping

parents:
[f2 -> c1e1f1: false
, a1 -> root: false
, e2c2 -> c1e1f1: false
, b1d1 -> a1: false
, c1e1f1 -> a1: false
]
children:
[a1 -> [b1d1: false, c1e1f1: false], root -> [a1: false], f2 -> [], e2c2 -> [], b1d1 -> [], c1e1f1 -> [f2: false, e2c2: false]]

In [28]:
val newEpa = mapping.buildNewEpa(epa)
newEpa

simple2.xescompressed
root,f2,a1,e2c2,b1d1,c1e1f1
b1d1,c1e1f1,a1,f2,e2c2
Transition(start=a1, activity=b1d1, end=b1d1),Transition(start=a1, activity=c1e1f1, end=c1e1f1),Transition(start=root, activity=a1, end=a1),Transition(start=c1e1f1, activity=f2, end=f2),Transition(start=c1e1f1, activity=e2c2, end=e2c2)




In [30]:
newEpa.states.map { it to (it as? State.PrefixState)?.from }

[(root, null), (f2, c1e1f1), (a1, root), (e2c2, c1e1f1), (b1d1, a1), (c1e1f1, a1)]